In [73]:
import pandas as pd
import numpy as np

def map_feature_portfolio(infile, metric='%score', mean_across_maps=False, filters={'test_pos': 2}):
    df = pd.read_csv(infile)
    df = df.rename(columns = {'strat':'portfolio'}) # renames the portfolio column

    # display(df.describe())
    
    #filters according to user criteria 
    for fname,fvalue in filters.items():
        if isinstance(fvalue, float):
            df = df[np.isclose(df[fname],fvalue)] 
        else:
            df = df[df[fname] == fvalue] 
        
    # display(df.describe())
    
    # human-friendly portfolio names
    df.replace({'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush+suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush+defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'}, inplace=True)

    # builds a hierarchical (multiindex) data frame (kinda groups by map, strategy & units)
    hier = df.set_index(['feature', 'mapname', 'portfolio', 'alpha', '_lambda', 'train_matches', 'dec_int', 'test_pos']) 
    #print(np.where(hier.index.duplicated()))  # finds entries with duplicate indexes, thx: https://stackoverflow.com/a/28652153/1251716
    
    # unstack the hierarchical dataframe up to the portfolio (all other parameters are grouped?)
    unstacked = hier.unstack(2)[metric]
    
    if mean_across_maps: 
        # if i group by the 'feature' level, i can get the mean on all maps played using that feature extractor
        return unstacked.groupby(level=['feature']).mean()
    else:
        return unstacked
    
# displays the results for all adversaries
opponents = ['A3N', 'SSSmRTS','StrategyTactics','PuppetSearchMCTS','WorkerRush','LightRush'] #A3N gives errors w/o budget =/

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100, 'train_matches': 100000, '_lambda': 0.5, 'alpha': 0.01,}

for o in opponents:
    print("---{}, no search budget---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_avg.csv'.format(o), '%score', True, filters))
    
    print("---{}, search budget=100ms---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_b100_avg.csv'.format(o), '%score', True, filters))
    

---A3N, no search budget---


portfolio,all,rush,rush+defense,rush+suport
feature,,,,
materialdistancehp,66.428571,53.359848,60.952381,99.583333
quadrantmodel,96.458333,60.773810,NaN,NaN


---A3N, search budget=100ms---


portfolio,all,rush
feature,,
materialdistancehp,NaN,33.583333
quadrantmodel,34.861111,NaN


---SSSmRTS, no search budget---


...

portfolio,all,rush
feature,,
materialdistancehp,NaN,47.333333
quadrantmodel,32.777778,NaN


---LightRush, no search budget---


portfolio,all,rush,rush+defense,rush+suport
feature,,,,
materialdistancehp,69.047619,64.166667,64.285714,51.041667
quadrantmodel,55.952381,57.142857,NaN,NaN


---LightRush, search budget=100ms---


portfolio,all,rush
feature,,
materialdistancehp,NaN,37.916667
quadrantmodel,32.222222,NaN


In [23]:
print("With search budget")
for o in opponents:
    print("---{}---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_b100_avg.csv'.format(o), '%score', True))

With search budget
---SSSmRTS---


portfolio,all,rush
feature,,
materialdistancehp,NaN,12.666667
quadrantmodel,13.75,NaN


---StrategyTactics---


portfolio,all,rush
feature,,
materialdistancehp,NaN,50.833333
quadrantmodel,37.222222,NaN


---PuppetSearchMCTS---


portfolio,rush
feature,
materialdistancehp,47.722222


---WorkerRush---


portfolio,all,rush
feature,,
materialdistancehp,NaN,47.333333
quadrantmodel,32.777778,NaN


---LightRush---


portfolio,all,rush
feature,,
materialdistancehp,NaN,37.916667
quadrantmodel,32.222222,NaN


In [20]:
### TEST TO RENAME THE PORTFOLIOS

df = pd.read_csv('../results/parameters/A3N_b100_avg.csv')
df = df[df.test_pos == 2] #filters to get the data for the average on both positions
df.replace({'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush_suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush_defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'}, inplace=True)
#strat.map( {'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush_suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush_defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'})
df

,num_reps,train_opp,test_opp,test_pos,mapname,feature,strat,reward,train_matches,dec_int,alpha,epsilon,gamma,_lambda,wins,draws,losses,test_matches,score,%score
2,3,selfplay,A3N,2,basesWorkers32x32A,materialdistancehp,rush,winlossdraw,100000,100,0.01,0.1,1,0.5,5.166667,0.166667,14.666667,20.0,5.250000,26.250000
5,3,selfplay,A3N,2,basesWorkers32x32A,quadrantmodel,all,winlossdraw,100000,100,0.01,0.1,1,0.5,3.666667,0.000000,16.333333,20.0,3.666667,18.333333
8,3,selfplay,A3N,2,BWDistantResources32x32,materialdistancehp,rush,winlossdraw,100000,100,0.01,0.1,1,0.5,9.833333,0.833333,9.333333,20.0,10.250000,51.250000
11,2,selfplay,A3N,2,(4)BloodBath.scmB,materialdistancehp,rush,winlossdraw,100000,100,0.01,0.1,1,0.5,2.500000,0.000000,17.500000,20.0,2.500000,12.500000
14,3,selfplay,A3N,2,basesWorkers8x8,quadrantmodel,all,winlossdraw,100000,100,0.01,0.1,1,0.5,14.000000,0.166667,5.833333,20.0,14.083333,70.416667
17,3,selfplay,A3N,2,basesWorkers16x16A,materialdistancehp,rush,winlossdraw,100000,100,0.01,0.1,1,0.5,5.333333,0.000000,14.666667,20.0,5.333333,26.666667
20,3,selfplay,A3N,2,basesWorkers16x16A,quadrantmodel,all,winlossdraw,100000,100,0.01,0.1,1,0.5,3.166667,0.000000,16.833333,20.0,3.166667,15.833333
23,3,selfplay,A3N,2,FourBasesWorkers8x8,materialdistancehp,rush,winlossdraw,100000,100,0.01,0.1,1,0.5,10.166667,0.166667,9.666667,20.0,10.250000,51.250000


In [62]:
isinstance(0.1, float)

True